# Part 1: Selecting a Function

<ul>
    <li> <code>df1.append(df2)</code>: stacking vertically
    <li> <code>pd.concat([df1,df2]) </code>: <b>*</b>
        <ul>
        <li> stacing many horizontally or vertically
        <li> simple inner/outter joins on indexes
        </ul>
    <li> <code>df1.join(df2):</code> inner/outer/left/right joins on indexes
    <li> <code>pd.merge([df1,df2])</code>: many joins in multiple columns  <b>*</b>
</ul>

<b>pd.merge and pd.concat is more flexable<b>

# Part 2: Joining (Merging) DataFrames


Using the <a href = "https://grouplens.org/datasets/movielens/"> MovieLense 100k data</a> let's create two DataFrames
<ul>
    <li><b>movies</b>: shows information about movies, namely a unique <b>movie_id</b> and it's <b>title</b>
    <li><b>ratings</b>: shows the <b>rating</b> that a particular <b>user_id</b> gave to a particular <b> movie_id</b> at a particular <b>timestamp</b>
</ul>

In [1]:
import pandas as pd

In [49]:
movie_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=movie_cols , encoding='latin-1', usecols=[0, 1], header=None)
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [50]:
movies.shape

(1682, 2)

In [51]:
movies.movie_id.nunique()

1682

In [52]:
movies["movie_id"].nunique()

1682

# Ratings

In [53]:
rating_cols = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings = pd.read_table("ml-100k/u.data", sep = "\t", header=None, names =rating_cols)
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [54]:
ratings.shape

(100000, 4)

In [55]:
ratings.movie_id.nunique()

1682

In [56]:
ratings.loc[ratings.movie_id==1, :].head()

,user_id,movie_id,rating,unix_timestamp
24,308,1,4,887736532
454,287,1,5,875334088
957,148,1,4,877019411
971,280,1,4,891700426
1324,66,1,3,883601324


In [57]:
# or 
ratings.loc[ratings["movie_id"]==1,:].head()

,user_id,movie_id,rating,unix_timestamp
24,308,1,4,887736532
454,287,1,5,875334088
957,148,1,4,877019411
971,280,1,4,891700426
1324,66,1,3,883601324


# Merging Movies and Ratings

Lets pretend that you want to exmine the rating Dataframe, but you want to know the <b> title</b> of each movie rather than its <b>movie_id </b>. The best way to accomplish this objective is by " joining" (or "merging") the DataFrames using Pandas <code>merge</code> function:

In [58]:
movies.columns

Index(['movie_id', 'title'], dtype='object')

In [59]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'unix_timestamp'], dtype='object')

We want to "Merge" the columns with promary key <b>movie_id</b>`

In [60]:
movie_ratings = pd.merge(movies, ratings)

In [61]:
movie_ratings.columns

Index(['movie_id', 'title', 'user_id', 'rating', 'unix_timestamp'], dtype='object')

In [62]:
movie_ratings.head(10)

,movie_id,title,user_id,rating,unix_timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
5,1,Toy Story (1995),5,4,875635748
6,1,Toy Story (1995),109,4,880563619
7,1,Toy Story (1995),181,3,878962392
8,1,Toy Story (1995),95,5,879197329
9,1,Toy Story (1995),268,3,875742341


In [63]:
movie_ratings.shape

(100000, 5)

Here's what just happened:

Pandas noticed that movies and ratings had one column in common, namely movie_id. This is the "key" on which the DataFrames will be joined.
The first movie_id in movies is 1. Thus, Pandas looked through every row in the ratings DataFrame, searching for a movie_id of 1. Every time it found such a row, it recorded the user_id, rating, and timestamp listed in that row. In this case, it found 452 matching rows.
The second movie_id in movies is 2. Again, Pandas did a search of ratings and found 131 matching rows.
This process was repeated for all of the remaining rows in movies.
At the end of the process, the movie_ratings DataFrame is created, which contains the two columns from movies (movie_id and title) and the three other colums from ratings (user_id, rating, and timestamp).

movie_id 1 and its title are listed 452 times, next to the user_id, rating, and timestamp for each of the 452 matching ratings.
movie_id 2 and its title are listed 131 times, next to the user_id, rating, and timestamp for each of the 131 matching ratings.
And so on, for every movie in the dataset.

In [64]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

(1682, 2)
(100000, 4)
(100000, 5)


Notice the shapes of the three DataFrames:

There are 1682 rows in the movies DataFrame.
There are 100000 rows in the ratings DataFrame.
The merge function resulted in a movie_ratings DataFrame with 100000 rows, because every row from ratings matched a row from movies.
The movie_ratings DataFrame has 5 columns, namely the 2 columns from movies, plus the 4 columns from ratings, minus the 1 column in common.
By default, the merge function joins the DataFrames using all column names that are in common (movie_id, in this case). The documentation explains how you can override this behavior.

# Part 3: What if...(Inner Join)


## What if the columns you want to join on don't have the same name?


In [65]:
movies.columns = ["m_id", "columns"]
movies.columns

Index(['m_id', 'columns'], dtype='object')

In [66]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'unix_timestamp'], dtype='object')

In [67]:
pd.merge(movies, ratings, left_on="m_id", right_on="movie_id")

,m_id,columns,user_id,movie_id,rating,unix_timestamp
0,1,Toy Story (1995),308,1,4,887736532
1,1,Toy Story (1995),287,1,5,875334088
2,1,Toy Story (1995),148,1,4,877019411
3,1,Toy Story (1995),280,1,4,891700426
4,1,Toy Story (1995),66,1,3,883601324
...,...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1678,1,889289570
99996,1679,B. Monkey (1998),863,1679,3,889289491
99997,1680,Sliding Doors (1998),863,1680,2,889289570
99998,1681,You So Crazy (1994),896,1681,3,887160722


## What if you want to join on one index?

In [73]:
movies = movies.set_index("m_id")
movies.head()

,columns
m_id,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [74]:
pd.merge(movies, ratings, left_index = True , right_on="movie_id")

,columns,user_id,movie_id,rating,unix_timestamp
24,Toy Story (1995),308,1,4,887736532
454,Toy Story (1995),287,1,5,875334088
957,Toy Story (1995),148,1,4,877019411
971,Toy Story (1995),280,1,4,891700426
1324,Toy Story (1995),66,1,3,883601324
...,...,...,...,...,...
75323,Mat' i syn (1997),863,1678,1,889289570
67302,B. Monkey (1998),863,1679,3,889289491
80394,Sliding Doors (1998),863,1680,2,889289570
92329,You So Crazy (1994),896,1681,3,887160722


## What if you join on two indexes?

In [79]:
ratings = ratings.set_index("movie_id")

KeyError: "None of ['movie_id'] are in the columns"

In [80]:
ratings.head()

,user_id,rating,unix_timestamp
movie_id,,,
242,196,3,881250949
302,186,3,891717742
377,22,1,878887116
51,244,2,880606923
346,166,1,886397596


In [81]:
pd.merge(movies,ratings, left_index = True, right_index= True)

,columns,user_id,rating,unix_timestamp
1,Toy Story (1995),308,4,887736532
1,Toy Story (1995),287,5,875334088
1,Toy Story (1995),148,4,877019411
1,Toy Story (1995),280,4,891700426
1,Toy Story (1995),66,3,883601324
...,...,...,...,...
1678,Mat' i syn (1997),863,1,889289570
1679,B. Monkey (1998),863,3,889289491
1680,Sliding Doors (1998),863,2,889289570
1681,You So Crazy (1994),896,3,887160722


# Part 4 : Four types of Joins

There are actually four types of joins supported in Pandas <code> merge() </code> function. Here's how they are described by the documentation:
    <ul>
        <li> <b>inner</b>: use intersection of keys for both frames, similar to SQL inner join; preserve the order of the left keys
        <li> <b>outer</b>: uses union of keys from both frames,simular to SQL full outer join; sort keys lexicographically
    <li> <b> left</b>: uses only keys from left frame, similar to SQL right outer join: reserve key order
    <li> <b> right</b>: uses only from the right frame, simular to SQL right outer join; preserve key order
    </ul>
    The default is the 'inner join', which was used when createing the movie_ratings DataFrame 
    <br>
    <br>
    It's easiest to understand the different types by look at some examples:
    <hr>

## Example DataFrame A and B

In [92]:
A = pd.DataFrame( {"color" : ["green", "yellow", "red"], "num": [1, 2, 3]})

In [93]:
A

,color,num
0,green,1
1,yellow,2
2,red,3


In [102]:
B = pd.DataFrame({"color": ["green", "yellow", "pink"], "size": ["s", "m", "l"]})
B

,color,size
0,green,s
1,yellow,m
2,pink,l


### Inner Join (A intercept B)

Only included obsercation found in both A and B:

In [97]:
pd.merge(A,B, how="inner")

,color,num,size
0,green,1,s
1,yellow,2,m


In [96]:
# or
inner_join = pd.merge(A, B, left_on="color", right_on="color")

In [95]:
inner_join

,color,num,size
0,green,1,s
1,yellow,2,m


### Outer Join (A union B) 

In [99]:
pd.merge(A,B, how="outer")

,color,num,size
0,green,1.0,s
1,yellow,2.0,m
2,red,3.0,NaN
3,pink,NaN,l


### Left Join (A includes intercept)

In [101]:
pd.merge(A, B, how="left")

,color,num,size
0,green,1,s
1,yellow,2,m
2,red,3,NaN


### Right Join (B includes intersect)

In [103]:
pd.merge(A, B, how="right")

,color,num,size
0,green,1.0,s
1,yellow,2.0,m
2,pink,NaN,l
